<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/main/Figure_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Specific yields by curves

In [ ]:
import matplotlib.pyplot as plt
import mizani
import numpy as np
import pandas as pd
import plotnine as gg
from matplotlib import colors
from scipy import interpolate

from IPython.display import display

In [ ]:
logistic_params = {
    '1.5 bil. ppl (logistic)': {
      'L': 5,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    },
    '1.0 bil. ppl (logistic)': {
      'L': 3.4,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    },
    '0.5 bil. ppl (logistic)': {
      'L': 1.9,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    }
}

linear_params = {
    '1.5 bil. ppl (linear)': {
      'Max': 2.6,  # L.kWh^-1
      'a': 0.026,
      'b': 0.0,
    },
    '1.0 bil. ppl (linear)': {
      'Max': 2.1,  # L.kWh^-1
      'a': 0.021,
      'b': 0.0,
    },
    '0.5 bil. ppl (linear)': {
      'Max': 1.6,  # L.kWh^-1
      'a': 0.016,
      'b': 0.0,
    }
}

df_yields = pd.DataFrame({
    'rH': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'SMAG_Zhao': [None, None, 0.21, None, None, 3.71, None, None, 9.28, None],
    'ZMW': [0.19, 0.24, 0.28, 0.33, 0.42, 0.44, 0.47, 0.5, 0.53, None],
    'Bagheri': [None, None, 0.27, None, None, 0.67, None, None, None, None],
    'MOFs': [None, 0.19, None, None, None, None, 0.84, None, None, None],
})

In [ ]:
df_tidy = pd.melt(df_yields,
        id_vars=['rH'],
        value_vars=['SMAG_Zhao', 'ZMW', 'Bagheri', 'MOFs'],
        var_name='device',
        value_name='yield')
df_tidy.head()

In [ ]:
df_fits = pd.DataFrame()
x = np.linspace(0, 100, num=200)
df_fits['rH'] = x

for name, params in logistic_params.items():
  df_fits[name] = params['L']/(1 + np.exp(-params['k']*(x- params['x0'])))

for name, params in linear_params.items():
  df_fits[name] = params['a']*x + params['b']

df_fits_tidy = pd.melt(df_fits,
                       id_vars=['rH'],
                       value_vars=list(logistic_params.keys()) + list(linear_params.keys()),
                       var_name='fit',
                       value_name='yield')
df_fits_tidy['fit_type'] = df_fits_tidy.fit.apply(lambda x: 'logistic' if 'logistic' in x else 'linear')

df_fits_tidy.head()

In [ ]:
from itertools import cycle

p = (gg.ggplot(gg.aes(x='rH'))
    + gg.geom_point(df_tidy, gg.aes(y='yield', shape='device'),
                    color='black', size=4, alpha=0.6, show_legend=True)
    + gg.geom_line(df_fits_tidy, gg.aes(y='yield', color='fit', linetype='fit_type'),
                   size=1, show_legend=True)
    + gg.scale_y_log10(limits=(0.1, 10))
    + gg.scale_x_continuous(breaks=np.linspace(0, 100, num=11),
                            limits=(10, 100))
    + gg.scale_color_manual(['#4a86e8', '#4a86e8', '#ff0101', '#ff0101', '#35a754', '#35a754'])
    + gg.theme_minimal()
    + gg.theme(figure_size=(8, 8))
    + gg.labs(title='Specific Yield by curve [L/kWh]',
              x='relative humidity',
              y='Specific Yield [L/kWh]')
)

print(p)